In [1]:
from pyhive import hive
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
import time
from collections import Counter
import numpy as np

In [2]:
# Read the data in from Hive and save to csv...if already have the csv, read it in

if 'time_series_data_reviews.csv' not in os.listdir():
    try:
        print('Getting data from Hive...')
        start = time.time()
        conn = hive.Connection(host = 'hadoop.rcc.uchicago.edu', 
                               port = 10000, 
                               username = 'mtoolsidas', 
                               database = 'dmp_yelp_rs')

        cursor = conn.cursor()

        query = 'select r.business_id,r.`date`,r.stars,b.simplified_category from review r join business_basicdata b on r.business_id = b.business_id'
        cursor.execute(query)
        res = cursor.fetchall()
        end = time.time()
        df = pd.DataFrame(res, columns = [i[0] for i in cursor.description])
        df.to_csv('time_series_data_reviews.csv', index = False)
        print('Finished.')
        print('Fetched data in {} seconds.'.format(end - start))
    except:
        print('ERROR: Connect to the VPN.')
        
else:
    print('Getting data from csv...')
    df = pd.read_csv('time_series_data_reviews.csv')
    print('Finished')

Getting data from csv...
Finished


In [3]:
# Checking data

df.head()

,r.business_id,r.date,r.stars,b.simplified_category
0,s5PTCyZGH5QGbVCnVdb6KA,2009-12-24 03:07:25,4.0,Italian
1,y9DmwlwW1TuEQWv9vfqp8Q,2010-06-29 18:17:17,2.0,American
2,emWWcSEYCEVdfbpoQh313Q,2017-05-24 17:14:09,3.0,Korean
3,2U_sNLFyXYr9iCob0FH_6w,2012-09-10 13:40:58,4.0,Pizza
4,KCl8BmcHa6Tgw48C0miNGg,2018-04-27 00:37:45,1.0,American


In [4]:
# Cleaning the column names to remove '_.'

df.columns = [c[2:] for c in df.columns]

In [5]:
# Converting `date` column to datetime and extracting date components

df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d %H:%M:%S')
df['just_date'] = df['date'].dt.date
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['day_name'] = df['date'].dt.day_name()
df['month_year'] = df['date'].dt.to_period('M').astype(str)
df['week'] = df['date'].dt.to_period('W').astype(str)

In [6]:
# Grouping data by category and month/year, getting the count of reviews and average stars for each group  

group_by_cat = df.groupby(['simplified_category','month_year']).agg({'business_id':'count', 'stars':'mean'})
group_by_cat = group_by_cat.rename(columns = {'business_id':'num_reviews'})
group_by_cat.index = group_by_cat.index.set_levels([group_by_cat.index.levels[0], pd.to_datetime(group_by_cat.index.levels[1])])

In [7]:
from statsmodels.tsa.seasonal import seasonal_decompose

dfs = []

# Looping through each category (index level 0) and performing decomposition analysis
for i in set(group_by_cat.index.get_level_values(0)):
    
    print('starting category {}...'.format(i))
    
    # refactoring df to have 1 index instead of multi-index (removing category index)
    tmp = group_by_cat.xs(i, level = 0, drop_level = True)
    # changing index (month_year) to monthly period
    tmp.index = tmp.index.to_period('M')
    # filling in any missing months in the data and interpolating the values for review count and average stars
    tmp = tmp.reindex(pd.period_range(tmp.index[0],tmp.index[-1],freq='M')).interpolate('time')
    # turning index back to datetime
    tmp.index = tmp.index.to_timestamp()
    # Performing decomposition using additive model (as magnitude of seasonlity does not change over time)
    # This is done for both review count and average stars
    decompose_num_reviews = seasonal_decompose(tmp.num_reviews, model = 'additive', extrapolate_trend = 'freq')
    decompose_avg_stars = seasonal_decompose(tmp.stars, model = 'additive', extrapolate_trend = 'freq')
    # Turning result into dataframe and appending to list of dfs
    tmp2 = pd.DataFrame(decompose_num_reviews.seasonal)
    tmp2 = tmp2.rename(columns = {'seasonal':'seasonal_num_reviews'})
    tmp2['simplified_category'] = i
    tmp2['trend_num_reviews'] = decompose_num_reviews.trend
    tmp2['seasonal_avg_stars'] = decompose_avg_stars.seasonal
    tmp2['trend_avg_stars'] = decompose_avg_stars.trend
    tmp2['residual_num_reviews'] = decompose_num_reviews.resid
    tmp2['residual_avg_stars'] = decompose_avg_stars.resid
    dfs.append(tmp2)

    print('finished with category {}.'.format(i))
    print('--- --- --- ---')

starting category Mediterranean...
finished with category Mediterranean.
--- --- --- ---
starting category Italian...
finished with category Italian.
--- --- --- ---
starting category Japanese...
finished with category Japanese.
--- --- --- ---
starting category Food Truck...
finished with category Food Truck.
--- --- --- ---
starting category Sandwiches...
finished with category Sandwiches.
--- --- --- ---
starting category Specialty Food...
finished with category Specialty Food.
--- --- --- ---
starting category Cafe...
finished with category Cafe.
--- --- --- ---
starting category seafood...
finished with category seafood.
--- --- --- ---
starting category Bar...
finished with category Bar.
--- --- --- ---
starting category Indian...
finished with category Indian.
--- --- --- ---
starting category Convenience Store...
finished with category Convenience Store.
--- --- --- ---
starting category Breakfast...
finished with category Breakfast.
--- --- --- ---
starting category Deli...
fi

In [9]:
# Concatenating dataframes for all categories into one and checking data

final_df = pd.concat(dfs)

In [24]:
final_df.index = final_df.index.strftime('%Y-%m')

In [25]:
# Save result

final_df.to_csv('time_series_cat_final.csv', index = True, index_label = 'month_year')

In [26]:
a1 = pd.read_csv('time_series_cat_final.csv')
a2 = pd.read_csv('time_series_cat_final2.csv')

In [28]:
a3 = a1.merge(a2, on = ['month_year','simplified_category'])

In [34]:
a3 = a3.rename(columns = {'seasonal_num_reviews':'seasonal_review_counts','trend_num_reviews':'trend_review_counts','residual_num_reviews':'residual_review_counts'})

In [35]:
a3

,month_year,seasonal_review_counts,simplified_category,trend_review_counts,seasonal_avg_stars,trend_avg_stars,residual_review_counts,residual_avg_stars,review_counts,avg_stars,is_pre_covid
0,2004-10,-10.937047,Mediterranean,-1.239753,0.046368,3.719984,17.176800,0.233648,5.0,4.000000,1
1,2004-11,-65.459662,Mediterranean,-0.081828,0.057082,3.744278,68.541490,0.698641,3.0,4.500000,1
2,2004-12,-78.359729,Mediterranean,1.076098,0.112814,3.768571,78.283632,1.118615,1.0,5.000000,1
3,2005-01,31.473047,Mediterranean,2.234023,0.011771,3.792865,-32.707070,0.595365,1.0,4.400000,1
4,2005-02,-16.314888,Mediterranean,3.391948,-0.036287,3.817158,13.922939,0.019129,1.0,3.800000,1
...,...,...,...,...,...,...,...,...,...,...,...
5751,2020-09,-320.464018,American,5925.692599,-0.002102,3.847879,3507.771419,-0.009390,9113.0,3.836388,0
5752,2020-10,-106.761719,American,5089.055944,-0.011415,3.855703,4501.705775,-0.006457,9484.0,3.837832,0
5753,2020-11,-745.703491,American,4252.419289,-0.014935,3.863527,4231.284202,0.038717,7738.0,3.887309,0
5754,2020-12,-901.000655,American,3415.782634,-0.030211,3.871351,4845.218021,0.036442,7360.0,3.877582,0


In [36]:
a4 = a3[['month_year','simplified_category','seasonal_review_counts','trend_review_counts','residual_review_counts','seasonal_avg_stars','trend_avg_stars','residual_avg_stars','review_counts','avg_stars','is_pre_covid']]

In [38]:
a4.to_csv('time_series_features.csv', index = False)

In [37]:
a4

,month_year,simplified_category,seasonal_review_counts,trend_review_counts,residual_review_counts,seasonal_avg_stars,trend_avg_stars,residual_avg_stars,review_counts,avg_stars,is_pre_covid
0,2004-10,Mediterranean,-10.937047,-1.239753,17.176800,0.046368,3.719984,0.233648,5.0,4.000000,1
1,2004-11,Mediterranean,-65.459662,-0.081828,68.541490,0.057082,3.744278,0.698641,3.0,4.500000,1
2,2004-12,Mediterranean,-78.359729,1.076098,78.283632,0.112814,3.768571,1.118615,1.0,5.000000,1
3,2005-01,Mediterranean,31.473047,2.234023,-32.707070,0.011771,3.792865,0.595365,1.0,4.400000,1
4,2005-02,Mediterranean,-16.314888,3.391948,13.922939,-0.036287,3.817158,0.019129,1.0,3.800000,1
...,...,...,...,...,...,...,...,...,...,...,...
5751,2020-09,American,-320.464018,5925.692599,3507.771419,-0.002102,3.847879,-0.009390,9113.0,3.836388,0
5752,2020-10,American,-106.761719,5089.055944,4501.705775,-0.011415,3.855703,-0.006457,9484.0,3.837832,0
5753,2020-11,American,-745.703491,4252.419289,4231.284202,-0.014935,3.863527,0.038717,7738.0,3.887309,0
5754,2020-12,American,-901.000655,3415.782634,4845.218021,-0.030211,3.871351,0.036442,7360.0,3.877582,0
